<a href="https://colab.research.google.com/github/ClaFlorez/Machine_Learning_Simplifie/blob/main/9_3_Projet_complet_de_systeme_de_recommandationipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Projet complet de système de recommandation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# ÉTAPE 1: COMPRÉHENSION BUSINESS
print("PROJET ML COMPLET: Système de Recommandation de Films")
print("=" * 80)

print("ÉTAPE 1: COMPRÉHENSION BUSINESS")
print("=" * 50)

print("Objectifs business définis:")
print("  • PRIMAIRE: Augmenter le temps de visionnage de 20%")
print("  • SECONDAIRE: Réduire le churn de 15%")
print("  • TERTIAIRE: Améliorer la satisfaction utilisateur")

print("\nMétriques de succès:")
print("  • Taux de clic sur recommandations > 15%")
print("  • Temps moyen de visionnage +20%")
print("  • Note de satisfaction > 4.2/5")
print("  • Réduction churn de 2.5% à 2.1%")

print("\nContraintes identifiées:")
print("  • Temps de réponse < 100ms")
print("  • Disponibilité 99.9%")
print("  • Respect RGPD sur données personnelles")
print("  • Coût infrastructure < 50k€/mois")

# ÉTAPE 2: COMPRÉHENSION DES DONNÉES
print(f"\nÉTAPE 2: COMPRÉHENSION DES DONNÉES")
print("=" * 50)

# Simuler des données réalistes de plateforme streaming
np.random.seed(42)
n_users = 10000
n_movies = 2000
n_interactions = 150000

print(f"Génération du dataset de streaming:")
print(f"  • Utilisateurs: {n_users:,}")
print(f"  • Films: {n_movies:,}")
print(f"  • Interactions: {n_interactions:,}")

# Créer les utilisateurs avec profils variés
users_data = []
for user_id in range(1, n_users + 1):
    age = np.random.normal(35, 12)
    age = np.clip(age, 16, 80)

    # Segmenter les utilisateurs
    if age < 25:
        segment = 'Jeunes'
        genres_pref = ['Action', 'Animation', 'Sci-Fi']
    elif age < 45:
        segment = 'Adultes'
        genres_pref = ['Drame', 'Comédie', 'Thriller']
    else:
        segment = 'Seniors'
        genres_pref = ['Drame', 'Romance', 'Documentaire']

    users_data.append({
        'user_id': user_id,
        'age': age,
        'segment': segment,
        'genres_preferes': np.random.choice(genres_pref, 2).tolist(),
        'abonnement_mois': np.random.exponential(12),
        'sessions_par_semaine': np.random.poisson(4) + 1
    })

df_users = pd.DataFrame(users_data)

# Créer les films
genres_disponibles = ['Action', 'Animation', 'Comédie', 'Drame', 'Horreur',
                     'Romance', 'Sci-Fi', 'Thriller', 'Documentaire']

movies_data = []
for movie_id in range(1, n_movies + 1):
    genre_principal = np.random.choice(genres_disponibles)
    annee = np.random.randint(1990, 2024)
    duree = np.random.normal(105, 20)  # 105 min moyenne
    duree = np.clip(duree, 60, 180)

    # Note moyenne (corrélée avec l'année et le genre)
    note_base = 3.5
    if annee > 2015:
        note_base += 0.3  # Films récents mieux notés
    if genre_principal in ['Drame', 'Documentaire']:
        note_base += 0.4  # Genres "sérieux" mieux notés

    note_moyenne = np.clip(note_base + np.random.normal(0, 0.8), 1, 5)

    movies_data.append({
        'movie_id': movie_id,
        'titre': f'Film_{movie_id:04d}',
        'genre_principal': genre_principal,
        'annee': annee,
        'duree_minutes': duree,
        'note_moyenne': note_moyenne,
        'nb_votes': np.random.poisson(500) + 50
    })

df_movies = pd.DataFrame(movies_data)

print(f"\nCaractéristiques des données:")
print(f"Utilisateurs:")
print(f"  • Âge moyen: {df_users['age'].mean():.1f} ans")
print(f"  • Abonnement moyen: {df_users['abonnement_mois'].mean():.1f} mois")
print(f"  • Sessions/semaine: {df_users['sessions_par_semaine'].mean():.1f}")

print(f"\nFilms:")
print(f"  • Année moyenne: {df_movies['annee'].mean():.0f}")
print(f"  • Durée moyenne: {df_movies['duree_minutes'].mean():.0f} min")
print(f"  • Note moyenne: {df_movies['note_moyenne'].mean():.2f}/5")

# Générer les interactions utilisateur-film
print(f"\nGénération des interactions utilisateur-film...")

interactions_data = []
for _ in range(n_interactions):
    user_id = np.random.randint(1, n_users + 1)
    movie_id = np.random.randint(1, n_movies + 1)

    # Récupérer les infos utilisateur et film
    user_info = df_users[df_users['user_id'] == user_id].iloc[0]
    movie_info = df_movies[df_movies['movie_id'] == movie_id].iloc[0]

    # Calculer la probabilité d'appréciation basée sur les préférences
    prob_like = 0.3  # Base

    # Ajuster selon les préférences de genre
    if movie_info['genre_principal'] in user_info['genres_preferes']:
        prob_like += 0.4

    # Ajuster selon l'âge et l'année du film
    age_movie = 2024 - movie_info['annee']
    if user_info['segment'] == 'Jeunes' and age_movie < 10:
        prob_like += 0.2
    elif user_info['segment'] == 'Seniors' and age_movie > 20:
        prob_like += 0.2

    # Ajuster selon la qualité du film
    if movie_info['note_moyenne'] > 4:
        prob_like += 0.3
    elif movie_info['note_moyenne'] < 2.5:
        prob_like -= 0.3

    # Générer l'interaction
    a_aime = np.random.random() < prob_like
    temps_visionne = np.random.uniform(0.1, 1.0) if a_aime else np.random.uniform(0.05, 0.4)

    interactions_data.append({
        'user_id': user_id,
        'movie_id': movie_id,
        'timestamp': datetime.now() - timedelta(days=np.random.randint(0, 365)),
        'a_aime': int(a_aime),
        'temps_visionne_pct': temps_visionne,
        'note_utilisateur': np.random.randint(1, 6) if a_aime else np.random.randint(1, 4)
    })

df_interactions = pd.DataFrame(interactions_data)

print(f"Interactions générées:")
print(f"  • Total interactions: {len(df_interactions):,}")
print(f"  • Taux d'appréciation: {df_interactions['a_aime'].mean():.1%}")
print(f"  • Temps moyen visionné: {df_interactions['temps_visionne_pct'].mean():.1%}")

# ÉTAPE 3: PRÉPARATION DES DONNÉES
print(f"\nÉTAPE 3: PRÉPARATION DES DONNÉES")
print("=" * 50)

# Créer les features pour la recommandation
# Joindre toutes les informations
df_ml = df_interactions.merge(df_users, on='user_id').merge(df_movies, on='movie_id')

# Encoder les variables catégorielles
df_encoded = pd.get_dummies(df_ml, columns=['segment', 'genre_principal'])

# Créer des features d'interaction
df_encoded['age_film'] = 2024 - df_encoded['annee']
df_encoded['ratio_duree_age'] = df_encoded['duree_minutes'] / df_encoded['age']
df_encoded['score_popularite'] = df_encoded['nb_votes'] / df_encoded['nb_votes'].max()

# Features finales pour le modèle
feature_cols = [col for col in df_encoded.columns if col not in
               ['user_id', 'movie_id', 'timestamp', 'a_aime', 'note_utilisateur',
                'titre', 'genres_preferes', 'temps_visionne_pct']]

X = df_encoded[feature_cols]
y = df_encoded['a_aime']  # Variable cible : l'utilisateur a-t-il aimé ?

print(f"Features préparées:")
print(f"  • Nombre de features: {len(feature_cols)}")
print(f"  • Exemples de features: {feature_cols[:10]}")

# Division des données
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDivision des données:")
print(f"  • Entraînement: {len(X_train):,} interactions")
print(f"  • Test: {len(X_test):,} interactions")
print(f"  • Taux d'appréciation train: {y_train.mean():.1%}")
print(f"  • Taux d'appréciation test: {y_test.mean():.1%}")

# ÉTAPE 4: MODÉLISATION
print(f"\nÉTAPE 4: MODÉLISATION")
print("=" * 50)

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entraîner le modèle de recommandation
model_recommandation = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

print("Entraînement du modèle de recommandation...")
model_recommandation.fit(X_train_scaled, y_train)

# ÉTAPE 5: ÉVALUATION
print(f"\nÉTAPE 5: ÉVALUATION")
print("=" * 50)

# Prédictions
y_pred = model_recommandation.predict(X_test_scaled)
y_pred_proba = model_recommandation.predict_proba(X_test_scaled)[:, 1]

# Métriques standard
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Performance du modèle:")
print(f"  • Précision: {precision:.3f} ({precision:.1%})")
print(f"  • Rappel: {recall:.3f} ({recall:.1%})")
print(f"  • F1-Score: {f1:.3f}")

# Métriques business spécifiques
# Simuler l'impact sur les KPIs business
taux_clic_baseline = 0.12  # 12% de taux de clic actuel
taux_clic_predit = precision * 0.8  # Estimation basée sur la précision

amelioration_engagement = (taux_clic_predit / taux_clic_baseline - 1) * 100
temps_visionne_additionnel = amelioration_engagement * 0.5  # 50% de l'amélioration se traduit en temps

print(f"\nImpact business estimé:")
print(f"  • Amélioration taux de clic: {amelioration_engagement:+.1f}%")
print(f"  • Temps de visionnage additionnel: {temps_visionne_additionnel:+.1f}%")

# Calcul du ROI
revenus_additionnels_par_user = 2.5  # 2.5€ par mois par utilisateur plus engagé
users_impactes = n_users * (amelioration_engagement / 100)
revenus_additionnels_annuels = users_impactes * revenus_additionnels_par_user * 12

cout_developpement = 200000  # 200k€ de développement
cout_infrastructure_annuel = 60000  # 60k€/an d'infrastructure

roi_net = revenus_additionnels_annuels - cout_infrastructure_annuel - cout_developpement
roi_pct = (roi_net / cout_developpement) * 100

print(f"\nROI du projet:")
print(f"  • Utilisateurs impactés: {users_impactes:,.0f}")
print(f"  • Revenus additionnels: {revenus_additionnels_annuels:,.0f}€/an")
print(f"  • Coût développement: {cout_developpement:,}€")
print(f"  • Coût infrastructure: {cout_infrastructure_annuel:,}€/an")
print(f"  • ROI première année: {roi_pct:.0f}%")

# ÉTAPE 6: DÉPLOIEMENT
print(f"\nÉTAPE 6: DÉPLOIEMENT")
print("=" * 50)

# Architecture de déploiement
print("Architecture de déploiement:")
print("  • API REST pour les recommandations")
print("  • Cache Redis pour les recommandations fréquentes")
print("  • Base de données pour stocker les interactions")
print("  • Pipeline de réentraînement automatique")
print("  • Monitoring en temps réel")

# Système de recommandation en action
def generer_recommandations(user_id, top_n=5):
    """
    Générer des recommandations pour un utilisateur
    """
    # Récupérer le profil utilisateur
    user_profile = df_users[df_users['user_id'] == user_id].iloc[0]

    # Obtenir les films non encore vus
    films_vus = df_interactions[df_interactions['user_id'] == user_id]['movie_id'].tolist()
    films_candidats = df_movies[~df_movies['movie_id'].isin(films_vus)]

    if len(films_candidats) == 0:
        return []

    # Créer les features pour chaque film candidat
    recommendations = []
    for _, movie in films_candidats.head(100).iterrows():  # Limiter pour performance
        # Simuler les features comme dans l'entraînement
        features_simulation = np.random.random(len(feature_cols))  # Simplification
        features_scaled = scaler.transform([features_simulation])

        # Prédire la probabilité d'appréciation
        proba_like = model_recommandation.predict_proba(features_scaled)[0, 1]

        recommendations.append({
            'movie_id': movie['movie_id'],
            'titre': movie['titre'],
            'genre': movie['genre_principal'],
            'note_moyenne': movie['note_moyenne'],
            'probabilite_appreciation': proba_like,
            'score_recommandation': proba_like * movie['note_moyenne']  # Score composite
        })

    # Trier par score et retourner le top N
    recommendations.sort(key=lambda x: x['score_recommandation'], reverse=True)
    return recommendations[:top_n]

# Test du système de recommandation
print(f"\nTest du système de recommandation:")
print("=" * 60)

# Tester sur 3 utilisateurs différents
users_test = [1, 500, 1000]

for user_id in users_test:
    user_info = df_users[df_users['user_id'] == user_id].iloc[0]
    recommendations = generer_recommandations(user_id, top_n=5)

    print(f"\nUtilisateur {user_id} ({user_info['segment']}, {user_info['age']:.0f} ans):")
    print(f"  Préférences: {user_info['genres_preferes']}")
    print(f"  Recommandations:")

    for i, rec in enumerate(recommendations, 1):
        print(f"    {i}. {rec['titre']} ({rec['genre']})")
        print(f"       Note: {rec['note_moyenne']:.1f}/5, Probabilité: {rec['probabilite_appreciation']:.1%}")

# Monitoring et amélioration continue
print(f"\nPlan de monitoring et amélioration:")
print("=" * 60)

print("MONITORING TEMPS RÉEL:")
print("  • Taux de clic sur recommandations")
print("  • Temps de réponse API")
print("  • Taux d'erreur système")
print("  • Satisfaction utilisateur (feedback)")

print("\nANALYSE HEBDOMADAIRE:")
print("  • Performance par segment utilisateur")
print("  • Évolution des préférences")
print("  • Détection de nouveaux patterns")
print("  • Analyse des films les plus/moins recommandés")

print("\nRÉENTRAÎNEMENT AUTOMATIQUE:")
print("  • Intégration des nouvelles interactions")
print("  • Mise à jour des profils utilisateurs")
print("  • Ajustement des paramètres")
print("  • Tests A/B pour valider les améliorations")

# Gestion des défis techniques
print(f"\nGestion des défis techniques:")
print("=" * 50)

print("PROBLÈME DU DÉMARRAGE À FROID:")
print("  • Nouveaux utilisateurs sans historique")
print("  • Solution: Recommandations basées sur démographie")
print("  • Questionnaire de préférences initial")

print("\nPROBLÈME DE SCALABILITÉ:")
print("  • Millions d'utilisateurs × millions de films")
print("  • Solution: Clustering des utilisateurs similaires")
print("  • Pré-calcul des recommandations populaires")

print("\nPROBLÈME DE DIVERSITÉ:")
print("  • Éviter les bulles de filtrage")
print("  • Solution: Injection de sérendipité")
print("  • Équilibre exploitation/exploration")

# Mesure du succès final
print(f"\nMesure du succès après 6 mois de déploiement:")
print("=" * 60)

# Simulation des résultats après déploiement
print("RÉSULTATS OBTENUS (simulation basée sur cas réels):")
print("  ✓ Taux de clic: 12% → 17% (+42%)")
print("  ✓ Temps de visionnage: +18% en moyenne")
print("  ✓ Churn: 2.5% → 2.2% (-12%)")
print("  ✓ Satisfaction: 3.8/5 → 4.1/5")
print("  ✓ ROI réalisé: 180% (supérieur aux prévisions)")

print("\nFACTEURS DE SUCCÈS IDENTIFIÉS:")
print("  • Implication des équipes produit dès le début")
print("  • Tests utilisateur approfondis")
print("  • Déploiement progressif avec monitoring")
print("  • Amélioration continue basée sur les retours")

print("\nLEÇONS APPRISES:")
print("  • L'expertise métier est aussi importante que la technique")
print("  • Les utilisateurs doivent sentir la valeur ajoutée")
print("  • La performance technique ne garantit pas le succès business")
print("  • La communication est clé pour l'adoption")

# Guide pour votre premier projet
print(f"\nGuide pour VOTRE premier projet ML:")
print("=" * 60)

print("1. CHOISISSEZ UN PROBLÈME SIMPLE:")
print("   • Objectif business clair et mesurable")
print("   • Données disponibles et de qualité")
print("   • Impact visible si succès")
print("   • Risque limité si échec")

print("\n2. COMMENCEZ PETIT:")
print("   • Prototype sur échantillon réduit")
print("   • Validation manuelle des premiers résultats")
print("   • Itération rapide")
print("   • Extension progressive")

print("\n3. IMPLIQUEZ LES PARTIES PRENANTES:")
print("   • Utilisateurs finaux dans la conception")
print("   • Validation des hypothèses métier")
print("   • Formation des équipes")
print("   • Communication régulière")

print("\n4. MESUREZ TOUT:")
print("   • Métriques techniques ET business")
print("   • Baseline avant déploiement")
print("   • Monitoring continu")
print("   • Tests A/B pour valider les améliorations")

print("\n5. PRÉPAREZ L'ÉCHEC:")
print("   • Plan B si les résultats ne sont pas au rendez-vous")
print("   • Critères d'arrêt définis à l'avance")
print("   • Apprentissage des échecs pour projets futurs")

print(f"\n🏆 FÉLICITATIONS!")
print("Vous avez maintenant toutes les clés pour réussir")
print("votre premier projet Machine Learning!")

PROJET ML COMPLET: Système de Recommandation de Films
ÉTAPE 1: COMPRÉHENSION BUSINESS
Objectifs business définis:
  • PRIMAIRE: Augmenter le temps de visionnage de 20%
  • SECONDAIRE: Réduire le churn de 15%
  • TERTIAIRE: Améliorer la satisfaction utilisateur

Métriques de succès:
  • Taux de clic sur recommandations > 15%
  • Temps moyen de visionnage +20%
  • Note de satisfaction > 4.2/5
  • Réduction churn de 2.5% à 2.1%

Contraintes identifiées:
  • Temps de réponse < 100ms
  • Disponibilité 99.9%
  • Respect RGPD sur données personnelles
  • Coût infrastructure < 50k€/mois

ÉTAPE 2: COMPRÉHENSION DES DONNÉES
Génération du dataset de streaming:
  • Utilisateurs: 10,000
  • Films: 2,000
  • Interactions: 150,000

Caractéristiques des données:
Utilisateurs:
  • Âge moyen: 35.3 ans
  • Abonnement moyen: 12.0 mois
  • Sessions/semaine: 5.0

Films:
  • Année moyenne: 2006
  • Durée moyenne: 105 min
  • Note moyenne: 3.67/5

Génération des interactions utilisateur-film...
Interactions g

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut


Utilisateur 1 (Adultes, 41 ans):
  Préférences: ['Thriller', 'Drame']
  Recommandations:
    1. Film_0097 (Action)
       Note: 4.4/5, Probabilité: 43.2%
    2. Film_0018 (Documentaire)
       Note: 4.7/5, Probabilité: 38.7%
    3. Film_0078 (Drame)
       Note: 4.4/5, Probabilité: 41.6%
    4. Film_0035 (Animation)
       Note: 4.1/5, Probabilité: 43.2%
    5. Film_0099 (Documentaire)
       Note: 5.0/5, Probabilité: 34.9%


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut


Utilisateur 500 (Jeunes, 24 ans):
  Préférences: ['Animation', 'Animation']
  Recommandations:
    1. Film_0023 (Action)
       Note: 5.0/5, Probabilité: 41.6%
    2. Film_0075 (Comédie)
       Note: 4.6/5, Probabilité: 42.4%
    3. Film_0078 (Drame)
       Note: 4.4/5, Probabilité: 41.2%
    4. Film_0073 (Drame)
       Note: 4.7/5, Probabilité: 38.0%
    5. Film_0063 (Comédie)
       Note: 4.5/5, Probabilité: 39.1%


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut


Utilisateur 1000 (Jeunes, 24 ans):
  Préférences: ['Action', 'Action']
  Recommandations:
    1. Film_0002 (Horreur)
       Note: 4.7/5, Probabilité: 42.0%
    2. Film_0060 (Documentaire)
       Note: 4.8/5, Probabilité: 40.7%
    3. Film_0099 (Documentaire)
       Note: 5.0/5, Probabilité: 38.7%
    4. Film_0018 (Documentaire)
       Note: 4.7/5, Probabilité: 39.5%
    5. Film_0082 (Documentaire)
       Note: 4.2/5, Probabilité: 44.3%

Plan de monitoring et amélioration:
MONITORING TEMPS RÉEL:
  • Taux de clic sur recommandations
  • Temps de réponse API
  • Taux d'erreur système
  • Satisfaction utilisateur (feedback)

ANALYSE HEBDOMADAIRE:
  • Performance par segment utilisateur
  • Évolution des préférences
  • Détection de nouveaux patterns
  • Analyse des films les plus/moins recommandés

RÉENTRAÎNEMENT AUTOMATIQUE:
  • Intégration des nouvelles interactions
  • Mise à jour des profils utilisateurs
  • Ajustement des paramètres
  • Tests A/B pour valider les améliorations

Gesti

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
